In [1]:
import friendlywords as fw
from XrayTo3DShape import CustomAutoEncoder, TLPredictor,VolumeAsInputExperiment,filter_wandb_run
import os

Failed to load image Python extension: /mnt/SSD0/mahesh-home/miniconda3/envs/xrayto3dbenchmark-cuda_11/lib/python3.8/site-packages/torchvision/image.so: undefined symbol: _ZN2at4_ops19empty_memory_format4callEN3c108ArrayRefIlEENS2_8optionalINS2_10ScalarTypeEEENS5_INS2_6LayoutEEENS5_INS2_6DeviceEEENS5_IbEENS5_INS2_12MemoryFormatEEE


2023-02-26 23:30:43,143 - Created a temporary directory at /tmp/tmppfr199zi
2023-02-26 23:30:43,145 - Writing /tmp/tmppfr199zi/_remote_module_non_scriptable.py


In [2]:
os.chdir('/mnt/SSD0/mahesh-home/xrayto3D-benchmark')

In [3]:
expt_dict = {'OneDConcat':'ParallelHeadsExperiment','MultiScale2DPermuteConcat':'ParallelHeadsExperiment','TwoDPermuteConcat':'ParallelHeadsExperiment','AttentionUnet':'VolumeAsInputExperiment',
'UNet':'VolumeAsInputExperiment'}

In [ ]:
train_csv_path = 'configs/paths/totalsegmentator_hips/TotalSegmentor-hips-DRR-full_train.csv'
test_csv_path = 'configs/paths/totalsegmentator_hips/TotalSegmentor-hips-DRR-full_val.csv'
gpu = 0
img_size = 128
resolution = 2.25
model_name = CustomAutoEncoder.__name__
unique_tag = fw.generate(1)
command = f'python train.py  {train_csv_path} {test_csv_path} --gpu {gpu} --tags   model-selection {unique_tag} --size {img_size} --batch_size 8 --accelerator gpu --res {resolution} --precision 16 --model_name {model_name} --epochs -1 --anatomy hip --loss BCEWithLogitsLoss  --lr 0.002 --steps 2500'
print(command)

model_name = TLPredictor.__name__
command = f'python train.py  {train_csv_path} {test_csv_path} --gpu {gpu} --tags   model-selection {unique_tag} --size {img_size} --batch_size 8 --accelerator gpu --res {resolution} --precision 16 --model_name {model_name} --epochs -1 --anatomy hip --loss DiceLoss  --lr 0.002 --steps 3000 '
runs = filter_wandb_run(anatomy='hip',tags=[unique_tag])
run_ids = [run.id for run in runs if run.config['model_name'] == CustomAutoEncoder.__name__]
print(run_ids)
assert len(run_ids) == 1, f'More than 1 runs found with tag {unique_tag}'
autoencoder_runid = run_ids[0]
print([run.id for run in runs])
if model_name == TLPredictor.__name__:
    command += f' --load_autoencoder_from runs/2d-3d-benchmark/{autoencoder_runid}/checkpoints/last.ckpt'
print(command)


In [13]:
for model_name,_ in expt_dict.items():
    unique_tag = fw.generate(1)

    command = f'python train.py  {train_csv_path} {test_csv_path} --gpu {gpu} --tags   model-selection {unique_tag} --size {img_size} --batch_size 8 --accelerator gpu --res {resolution} --precision 16 --model_name {model_name} --epochs -1 --anatomy hip --loss DiceLoss  --lr 0.002 --steps 3000 '
    print(command,'\n')

python train.py  configs/paths/totalsegmentator_hips/TotalSegmentor-hips-DRR-full_train.csv configs/paths/totalsegmentator_hips/TotalSegmentor-hips-DRR-full_val.csv --gpu 0 --tags   model-selection card --size 128 --batch_size 8 --accelerator gpu --res 2.25 --precision 16 --model_name OneDConcat --epochs -1 --anatomy hip --loss DiceLoss  --lr 0.002 --steps 3000  

python train.py  configs/paths/totalsegmentator_hips/TotalSegmentor-hips-DRR-full_train.csv configs/paths/totalsegmentator_hips/TotalSegmentor-hips-DRR-full_val.csv --gpu 0 --tags   model-selection vest --size 128 --batch_size 8 --accelerator gpu --res 2.25 --precision 16 --model_name MultiScale2DPermuteConcat --epochs -1 --anatomy hip --loss DiceLoss  --lr 0.002 --steps 3000  

python train.py  configs/paths/totalsegmentator_hips/TotalSegmentor-hips-DRR-full_train.csv configs/paths/totalsegmentator_hips/TotalSegmentor-hips-DRR-full_val.csv --gpu 0 --tags   model-selection mallet --size 128 --batch_size 8 --accelerator gpu --